# Main code

In [12]:
from classes import Species, Observation


        
        
if __name__ == "__main__":
    global species
    sc.addPyFile("classes.py")

    data = sc.textFile("/home/WUR/steen176/Downloads/test.dat")
    #data = sc.textFile('hdfs://scomp1334:9000/user/steen176/csv/butterflies.csv')

    corrected = data.map(lambda line: correctLine(line))
    species = extractSpecies(corrected)
    
    
    #rdd.map did not work here
    map(getObservation, corrected.toLocalIterator())
    
    to_cluster = sc.parallelize(species.keys())
    print(type(to_cluster))
    
    #how i want it
    #clusters = to_cluster.map(lambda line: cluster(line))
    
    #what works
    clusters = map(cluster, species.keys())
    print clusters[0].clusterCenters
    
    

    
    print findCombinations(clusters[0], centers_all)


    

<class 'pyspark.rdd.RDD'>
[array([ 90.076225  ,   1.36562839]), array([ 90.076225  ,   1.36562839]), array([ 90.076225  ,   1.36562839]), array([ 90.076225  ,   1.36562839])]


# Utility

In [2]:
def getSpecies(line):
    line = line.split(',')
    return line[0]

def correctLine(line):
    line = line.split(',')
    if len(line) != 10:
        return ','.join(line[0:4] + [" ".join(line[4:6])]+ line[6:])
    else:
        return ','.join(line)

def getObservation(line):
    line = line.split(',')
    specie = line[0]
    for i in xrange(int(line[5])):
        longti = line[8]
        lati = line[9]
        obs = Observation(longti,lati)
        species[specie].addObservation(obs)
    
    
def createSpecieObjects(species_list):
    species_objects = {}
    for specie in species_list:
        obj = Species(specie)
        species_objects[specie] = obj
    return species_objects


def extractSpecies(rdd):
    species = rdd.map(getSpecies)
    species_list  = list(set([x for x in species.toLocalIterator()]))
    return createSpecieObjects(species_list)


# Clustering

In [3]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import SparseVector
from math import sqrt
NUM_CLUSTER = 4

def kmeans(sparse_vectors):
    #sparsevectors should be a rdd containing sparsevectors
    return KMeans.train(sparse_vectors, NUM_CLUSTER, maxIterations=20, runs=100, initializationMode="random")

def cluster(specie):
    
    vectors = species[specie].getVectorRDD()
    return kmeans(sc.parallelize(vectors))


    
def error(point):
    center = clusters.centers[clusters.predict(point)]
    denseCenter = DenseVector(numpy.ndarray.tolist(center))
    return sqrt(sum([x**2 for x in (DenseVector(point.toArray()) - denseCenter)]))

    WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
    print(WSSSE)

# Comparing clusters

In [6]:
from itertools import combinations
import numpy 
def centerdiffs(centers, DIFF=0.1):
    centerA = centers[0]
    centerB = centers[1]
    
    longA = centerA[0]
    longB = centerB[0]
    lattA = centerA[1]
    lattB = centerB[1]
    
    longDiff = numpy.subtract(longA, longB)
    lattDiff = numpy.subtract(lattA, lattB)
    
    if longDiff < DIFF and lattDiff < DIFF:
        return True
    else:
        return False
    
def reduceRare(rare_specie_centers, changed=True):
    #probably don't use
    if not changed:
        return rare_specie_centers
    
    for i in xrange(len(rare_specie_centers)-1):
        one = rare_specie_centers[i]
        two = rare_specie_centers[i+1]


def findCombinations(rare_specie_centers, all_species_centers):

    for one in all_species_centers:
    #    for center in rare_specie_centers:
     #       pass

from pyspark.mllib.linalg import SparseVector

class Species():
    def __init__(self, name):
        self.name = name
        self.observations = []

    def addObservation(self, observ):
        self.observations.append(observ)

    def getVectorRDD(self):
        return [x.toSparseVector() for x in self.observations]

class Observation():
    def __init__(self,longti, lati):
        self.longti = longti
        self.lati = lati

    def __str__(self):
        return "observatio"

    def __repr__(self):
        return "observation"

    def toSparseVector(self):
        return SparseVector(2, [[0, self.longti], [1, self.lati]])